# EVWHS Raw NTF Strips to MS 4-band 2m ortho TOA reflectance
1. Clone PGC imagery_utils repo
1. Edit lib/ortho_functions.py omax setting for Int16 reflectance 'rf' mode: VNIR (from 2000.0 to 10000.0) and SWIR/CAVIS (16000.0 to 10000.0)
1. Create and activate PGC conda env
1. Create list of multispectral NTF files
1. Generate 4-band TOA orthos for each sub-image

1. Choose latest gdal env for notebook (cog)

In [1]:
import os
import glob
import subprocess
from osgeo import gdal
from datetime import datetime
from collections import Counter
import numpy as np
import pandas as pd
import pathlib

from platform import python_version
print(python_version())


3.9.21


In [2]:
# Test running subprocess gdal in a specific conda env
import subprocess

conda_path = "/home/mmacander/miniconda3/bin/conda"
conda_env = "cog"  # Change this to your environment name

# subprocess.run(["conda", "run", "-n", conda_env, "gdalwarp", "--version"], check=True)

warp_cmd = [
    conda_path, "run", "-n", conda_env,
    "gdalwarp",
    "--version"]

# Print and execute the command
print(f"🚀 Running: {' '.join(warp_cmd)}")
subprocess.run(warp_cmd, check=True)


🚀 Running: /home/mmacander/miniconda3/bin/conda run -n cog gdalwarp --version
GDAL 3.10.1, released 2025/01/08



CompletedProcess(args=['/home/mmacander/miniconda3/bin/conda', 'run', '-n', 'cog', 'gdalwarp', '--version'], returncode=0)

In [3]:
ortho_subimage_dir = "/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/ortho_pgc"
os.chdir(ortho_subimage_dir)

output_dir = "/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/ortho_toa_strips"
pathlib.Path(os.path.join(output_dir, "vrt")).mkdir(parents=True, exist_ok=True)


subimages = glob.glob(ortho_subimage_dir + "/*.tif")
print(subimages[:10])


['/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/ortho_pgc/23JUL21210509-M1BS-050296456010_01_P008_u16rf3338.tif', '/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/ortho_pgc/23JUL21205720-M1BS-050296474010_01_P010_u16rf3338.tif', '/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/ortho_pgc/21AUG30204810-M1BS-050296461010_01_P005_u16rf3338.tif', '/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/ortho_pgc/23JUL26212327-M1BS-050296496010_01_P005_u16rf3338.tif', '/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/ortho_pgc/24JUN29212914-M1BS-050296500010_01_P008_u16rf3338.tif', '/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/ortho_pgc/23JUL21210510-M1BS-050296456010_01_P007_u16rf3338.tif', '/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/ortho_pgc/24JUN29212846-M1BS-050296485010_01_P001_u16rf3338.tif', '/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/ortho_pgc/13JUN18214810-M1BS-050296499010_01_P002_u16rf3338.tif', '/data/gis/raster_base/Alaska/AKVegMap/

In [4]:
# Extract filename from path and split on dashes/underscores
data = []
for path in subimages:
    filename = os.path.basename(path)  # Get file name from path
    name_parts = [p for part in filename.split('_') for p in part.split('-')]  # Split on _ and -
    order_id = f"{name_parts[2]}_{name_parts[3]}"
    data.append([path] + [filename] + [order_id])  # Store filename + split parts

# Create DataFrame with dynamic column names
# max_splits = max(len(row) for row in data)
# print(max_splits)

columns = ["input_path"] + ["input_file"] + ["order_id"]# [f"Part_{i+1}" for i in range(max_splits - 2)]
# print(data)
# print(columns)

df = pd.DataFrame(data, columns=columns)

# print(df)

# Group by order_id and aggregate input_path as a list
order_grouped_df = df.groupby("order_id")["input_path"].agg(list).reset_index()
print(order_grouped_df.head())

          order_id                                         input_path
0  050296456010_01  [/data/gis/raster_base/Alaska/AKVegMap/EVWHS/m...
1  050296457010_01  [/data/gis/raster_base/Alaska/AKVegMap/EVWHS/m...
2  050296458010_01  [/data/gis/raster_base/Alaska/AKVegMap/EVWHS/m...
3  050296459010_01  [/data/gis/raster_base/Alaska/AKVegMap/EVWHS/m...
4  050296460010_01  [/data/gis/raster_base/Alaska/AKVegMap/EVWHS/m...


In [5]:
# Define search pattern to find all .ntf files in subdirectories
# ntf_files = glob.glob(order_dir + "*_MUL/*.NTF", recursive=True)
ntf_files = order_grouped_df.iloc[0]["input_path"]

# ntf_files = glob.glob("**/*.ntf", recursive=True)
# output_cog = "/data/gis/raster_base/Alaska/AKVegMap/EVWHS/nome_beaver/outputs_python/final_mosaic_cog4.tif "  # Output COG filename

# Ensure we found NTF files
if not ntf_files:
    raise ValueError("❌ No .ntf files found in subdirectories!")

print(f"✅ Found {len(ntf_files)} NTF files to process.")
print(ntf_files)

ntf_file = ntf_files[0]
print("\n", ntf_file)

✅ Found 9 NTF files to process.
['/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/ortho_pgc/23JUL21210509-M1BS-050296456010_01_P008_u16rf3338.tif', '/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/ortho_pgc/23JUL21210510-M1BS-050296456010_01_P007_u16rf3338.tif', '/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/ortho_pgc/23JUL21210513-M1BS-050296456010_01_P004_u16rf3338.tif', '/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/ortho_pgc/23JUL21210517-M1BS-050296456010_01_P001_u16rf3338.tif', '/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/ortho_pgc/23JUL21210511-M1BS-050296456010_01_P006_u16rf3338.tif', '/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/ortho_pgc/23JUL21210512-M1BS-050296456010_01_P005_u16rf3338.tif', '/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/ortho_pgc/23JUL21210508-M1BS-050296456010_01_P009_u16rf3338.tif', '/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/ortho_pgc/23JUL21210515-M1BS-050296456010_01_P003_u16rf3338.tif', '/data/

In [6]:
# Examine one file

# Open the NTF file
dataset = gdal.Open(ntf_file)
if not dataset:
    raise ValueError(f"❌ Could not open {ntf_file}")

# Get general metadata
metadata = dataset.GetMetadata()
print("✅ NTF Metadata:")
for key, value in metadata.items():
    print(f"{key}: {value}")

# Get the number of bands
num_bands = dataset.RasterCount
print(f"📌 Number of bands: {num_bands}")

# Get geotransform (spatial metadata)
geotransform = dataset.GetGeoTransform()
print(f"📌 Geotransform: {geotransform}")

# Get projection (Coordinate Reference System)
projection = dataset.GetProjection()
print(f"📌 Projection: {projection}")

# Get dimensions
width = dataset.RasterXSize
height = dataset.RasterYSize
print(f"📌 Dimensions: {width} x {height} pixels")

# Get information about each band
for i in range(1, num_bands + 1):
    band = dataset.GetRasterBand(i)
    print(f"📌 Band {i}: Type={gdal.GetDataTypeName(band.DataType)}, NoDataValue={band.GetNoDataValue()}")

# Close the dataset
dataset = None

NITF_ISORCE = metadata['NITF_ISORCE']
print(NITF_ISORCE)


✅ NTF Metadata:
NITF_ABPP: 11
NITF_CCS_COLUMN: 0
NITF_CCS_ROW: 0
NITF_CLEVEL: 06
NITF_ENCRYP: 0
NITF_FBKGC: 126,126,126
NITF_FDT: 20250221024806
NITF_FHDR: NITF02.10
NITF_FSCLAS: U
NITF_FSCLSY: US
NITF_FSCOP: 00000
NITF_FSCPYS: 00000
NITF_FTITLE: 23JUL21210509-M1BS-050296456010_01_P008.NTF
NITF_IALVL: 0
NITF_IC: NC
NITF_ICAT: MS
NITF_ICORDS: G
NITF_IDATIM: 20230721210510
NITF_IDLVL: 1
NITF_IGEOLO: 631139N1443650W631311N1441302W630447N1441258W630247N1443726W
NITF_IID1: M1EA9D8000
NITF_IID2: 21JUL23WV021400023JUL21210509-M1BS-050296456010_01_P008
NITF_ILOC_COLUMN: 0
NITF_ILOC_ROW: 0
NITF_IMAG: 1.0 
NITF_IMAGE_COMMENTS: The imagery and metadata data on this media is the property of                  Maxar Technologies and is licensed for use only.                                All use must be in accordance with the terms of the license that                accompanies the media. If the license is purchased under contract               use is in accordance with the license therein          

/home/mmacander/miniconda3/envs/cog/lib/python3.9/site-packages/osgeo/gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(
Warning 1: PROJ: proj_create_from_database: Open of /home/mmacander/miniconda3/envs/cog/share/proj failed


In [7]:
# Test extraction of catalog id
# Ensure we have NTF files
if not ntf_files:
    raise ValueError("❌ No .ntf files found!")

print(f"✅ Found {len(ntf_files)} NTF files.")

# Loop through each NTF file to extract the `NITF_PIAIMC_SOURCE` metadata
for ntf_file in ntf_files:
    dataset = gdal.Open(ntf_file)
    if dataset:
        metadata = dataset.GetMetadata()
        source_metadata = metadata.get("NITF_PIAIMC_SOURCE", "Unknown")
        
        # Check if the source contains "VNIR"
        # if "VNIR" in source_info:
        #     print(f"📌 {ntf_file}: {source_info}")
        # vnir_str = source_info.get("VNIR", "VNIR key not found")
    # Check if VNIR exists inside the metadata string
    if "VNIR:" in source_metadata:
        for line in source_metadata.split("\n"):  # Handle multi-line metadata
            if line.startswith("VNIR:"):
                vnir_value = line.split(":", 1)[1].strip()  # Get value after colon
       
        print(vnir_value)
        # return(vnir_value)
        dataset = None  # Close dataset



✅ Found 9 NTF files.
10300100EA9D8000
10300100EA9D8000
10300100EA9D8000
10300100EA9D8000
10300100EA9D8000
10300100EA9D8000
10300100EA9D8000
10300100EA9D8000
10300100EA9D8000


In [ ]:
# Run on batch of images from one order
for index, row in order_grouped_df.iterrows():
    print(row)
    print(f"\nOrder id: {row['order_id']}")
    # Define search pattern to find all .ntf files in subdirectories
    # ntf_files = glob.glob(order_dir + "*_MUL/*.NTF", recursive=True)
    ntf_files = row["input_path"]
    
    # Ensure we have files
    if not ntf_files:
        # raise ValueError("❌ No .ntf files found!")
        print("❌ No .ntf files found!")
        continue
    
    print(f"✅ Found {len(ntf_files)} NTF files.")
    
    # Store metadata values
    metadata_values = {}
    
    # Extract metadata from each file
    for ntf_file in ntf_files:
        dataset = gdal.Open(ntf_file)
        if dataset:
            metadata = dataset.GetMetadata()
            for key, value in metadata.items():
                if value and value.strip():  # Remove empty or whitespace-only values
                    if key not in metadata_values:
                        metadata_values[key] = []
                    metadata_values[key].append(value.strip())
            dataset = None  # Close file
    
    # Function to aggregate metadata
    aggregated_metadata = {}
    for key, values in metadata_values.items():
        try:
            # If the key ends with "_DATE", use the minimum date
            if key.upper().endswith("_DATE"):
                aggregated_metadata[key] = min(values)
            else:
                # Convert values to float if possible, then take the median
                numeric_values = [float(v) for v in values if v.replace('.', '', 1).isdigit()]
                if numeric_values:
                    aggregated_metadata[key] = str(np.median(numeric_values))
                else:
                    # Take the lexicographical minimum for non-numeric values
                    aggregated_metadata[key] = min(values)
        except ValueError:
            # Take minimum for categorical values
            aggregated_metadata[key] = min(values)
    
    # print("✅ Aggregated Metadata:")
    # for k, v in aggregated_metadata.items():
    #     print(f"{k}: {v}")
    
    # source_metadata = aggregated_metadata.get("NITF_PIAIMC_SOURCE", "Unknown")
    source_metadata = aggregated_metadata.get("NITF_PIAIMC_SOURCE", "Unknown")
        
        # Check if the source contains "VNIR"
        # if "VNIR" in source_info:
        #     print(f"📌 {ntf_file}: {source_info}")
        # vnir_str = source_info.get("VNIR", "VNIR key not found")
    # Check if VNIR exists inside the metadata string
    if "VNIR:" in source_metadata:
        for line in source_metadata.split("\n"):  # Handle multi-line metadata
            if line.startswith("VNIR:"):
                catid = line.split(":", 1)[1].strip()  # Get value after colon
       
        print(catid)
    
    sensor = aggregated_metadata.get("NITF_ISORCE")
    
    # Find the minimum date
    # if acquisition_dates:
    #     min_date = min(acquisition_dates)  # Dates are stored as strings, lexicographically sorted works for YYYYMMDD
    #     max_date = max(acquisition_dates)  # Dates are stored as strings, lexicographically sorted works for YYYYMMDD
    #     print(f"✅ Earliest acquisition date: {min_date}")
    #     print(f"✅   Latest acquisition date: {max_date}")
    # else:
    #     print("❌ No acquisition dates found in metadata!")
    
    min_acquisition_date = aggregated_metadata.get("NITF_STDIDC_ACQUISITION_DATE")
    min_acquisition_date = f"{min_acquisition_date[:8]}_{min_acquisition_date[8:]}"
    print(f"✅ Earliest acquisition date: {min_acquisition_date}")
    
    # # If only one unique VNIR value exists, print it
    # if len(vnir_counter) == 1:
    #     catid = next(iter(vnir_counter))  # Get the single VNIR value
    #     print(f"Catalog ID: {catid}")
    # else:
    #     print("Multiple catalog ids, exiting.")
    #     sys.exit()  # Exit without printing anything
    print(f"Catalog ID: {vnir_value}")
    
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    
    output_base = f"MS4_2m_{min_acquisition_date}_{sensor}_{catid}_v{timestamp}"
    output_vrt = os.path.join(output_dir, "vrt", f"{output_base}.vrt")
    output_cog = os.path.join(output_dir, f"{output_base}.tif")
    
    print(f"Output COG Filename: {output_cog}")
    
    # # Function to determine the number of bands in an NTF file
    # def get_num_bands(ntf_file):
    #     dataset = gdal.Open(ntf_file)
    #     if dataset:
    #         num_bands = dataset.RasterCount
    #         dataset = None  # Close the dataset
    #         return num_bands
    #     else:
    #         raise ValueError(f"❌ Could not open {ntf_file}")
    
    # # Check the band count of the first NTF file (assumes all have the same structure)
    # num_bands = get_num_bands(ntf_files[0])
    # print(f"✅ Detected {num_bands} bands.")
    
    # # Define which bands to process
    # bands_to_process = list(range(1, 5)) if num_bands == 4 else [2, 3, 5, 7]
    
    # # Apply aggregated metadata to output COG
    # output_cog = os.path.join(output_dir, output_name)  # Output COG filename
    
    # gdal.SetConfigOption("GDAL_TIFF_INTERNAL_MASK", "YES")  # Enable internal mask support
    gdal.SetConfigOption("GDAL_CACHEMAX", "1536")
    
    vrt_options = gdal.BuildVRTOptions(
        separate=False
    )
    
    translate_options = gdal.TranslateOptions(
        format="COG",  # Output format is COG
        creationOptions=["NUM_THREADS=20",
                         "PREDICTOR=2",  # Apply LZW compression predictor
                         "BIGTIFF=IF_SAFER",  # Enable BigTIFF support
                         "OVERVIEW_RESAMPLING=AVERAGE",
                         "OVERVIEWS=IGNORE_EXISTING"],
        stats=False
    )
    
    # warp_options = gdal.WarpOptions(
    #     dstSRS="EPSG:3338",  # Set target spatial reference
    #     srcBands=bands_to_process,
    #     multithread=True,
    #     warpOptions=["NUM_THREADS=20"],
    #     creationOptions=["NUM_THREADS=20",
    #                      "PREDICTOR=2",  # Apply LZW compression predictor
    #                      "BIGTIFF=IF_SAFER"],  # Enable BigTIFF support
    #     # "--config", "GDAL_CACHEMAX 1536",
    #     warpMemoryLimit=1536,
    #     rpc=True,  # Use Rational Polynomial Coefficients
    #     transformerOptions=[f"RPC_DEM={dem_tif}"],  # Use the specified DEM
    #     srcNodata=0,
    #     format="COG",  # Output format is COG
    #     xRes=2,
    #     yRes=2,
    #     targetAlignedPixels=True,
    #     resampleAlg="cubic",  # Resampling method
    #     # metadataOptions=aggregated_metadata  # Apply aggregated metadata
    #     copyMetadata=True)
    
    # # Run gdal.Warp with aggregated metadata
    # gdal.Warp(output_cog, ntf_files, options=warp_options)

    gdal.BuildVRT(
        output_vrt,
        row['input_path'],
        options=vrt_options
    )

    gdal.Translate(
        output_cog,
        output_vrt,
        options=translate_options
    )
    
    # # Construct the gdalinfo command
    info_cmd = [
        conda_path, "run", "-n", conda_env,
        # "gdalwarp",
        "/home/mmacander/miniconda3/envs/cog/bin/gdalinfo",
        "-stats",
        "-hist"
    ]
    
    info_cmd.extend([output_cog])
    
    # # Print and execute the command
    print(f"🚀 Running: {' '.join(info_cmd)}")
    # subprocess.run(info_cmd, shell=True, executable="/bin/bash", check=True)
    subprocess.run(info_cmd, check=True)
    
    print(f"✅ Orthorectified COG saved with metadata, stats: {output_cog}")


order_id                                        050296456010_01
input_path    [/data/gis/raster_base/Alaska/AKVegMap/EVWHS/m...
Name: 0, dtype: object

Order id: 050296456010_01
✅ Found 9 NTF files.
10300100EA9D8000
✅ Earliest acquisition date: 20230721_210509
Catalog ID: 10300100EA9D8000
Output COG Filename: /data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/ortho_toa_strips/MS4_2m_20230721_210509_WV02_10300100EA9D8000_v20250228102438.tif


Warning 1: PROJ: proj_create_from_database: Open of /home/mmacander/miniconda3/envs/cog/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /home/mmacander/miniconda3/envs/cog/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /home/mmacander/miniconda3/envs/cog/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /home/mmacander/miniconda3/envs/cog/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /home/mmacander/miniconda3/envs/cog/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /home/mmacander/miniconda3/envs/cog/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /home/mmacander/miniconda3/envs/cog/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /home/mmacander/miniconda3/envs/cog/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /home/mmacander/miniconda3/envs/cog/share/proj failed
ERROR 1: PROJ: proj_create: no database contex